# Segmenting Toronto
Coursera Data Science Capstone, part project

In [28]:
# import libraries
import requests         # getting HTML content
import lxml.html as lh  # parsing relevant fields
import pandas as pd

Scraping the data from the Wikipedia website follows https://towardsdatascience.com/web-scraping-html-tables-with-python-c9baba21059. So I do not use BeatifulSoup here as proposed on coursera.org. However, the result is the same.

In [29]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
#Create a handle, page, to handle the contents of the website
page = requests.get(url)
#Store the contents of the website under doc
doc = lh.fromstring(page.content)
#Parse data that are stored between <tr>..</tr> of HTML
tr_elements = doc.xpath('//tr')

Check the length of the first 12 rows for sanity (check if all rows have the same width)

In [30]:
[len(T) for T in tr_elements[:12]]

[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]

Parse the table header

In [31]:
tr_elements = doc.xpath('//tr')
#Create empty list
col=[]
i=0
#For each row, store each first element (header) and an empty list
for t in tr_elements[0]:
    i+=1
    name=t.text_content()
    name=name.replace("\n","")
    print('%d:"%s"'%(i,name))
    col.append((name,[]))

1:"Postcode"
2:"Borough"
3:"Neighbourhood"


Now create a pandas dataframe

In [32]:
#Since out first row is the header, data is stored on the second row onwards
for j in range(1,len(tr_elements)):
    #T is our j'th row
    T=tr_elements[j]
    
    #If row is not of size 10, the //tr data is not from our table 
    if len(T)!=10:
        break
    
    #i is the index of our column
    i=0
    
    #Iterate through each element of the row
    for t in T.iterchildren():
        data=t.text_content() 
        #Check if row is empty
        if i>0:
        #Convert any numerical value to integers
            try:
                data=int(data)
            except:
                pass
        #Append the data to the empty list of the i'th column
        col[i][1].append(data)
        #Increment i for the next column
        i+=1

Let’s check the length of each column (should all be the same)

In [33]:
[len(C) for (title,C) in col]

[0, 0, 0]